In [2]:
from dotenv import load_dotenv
load_dotenv()


True

In [5]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI

loader = WebBaseLoader("https://en.wikipedia.org/wiki/Parul_University")
docs = loader.load()
docs


[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Parul_University', 'title': 'Parul University - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nParul University - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nHistory, gove

In [6]:
from langchain_text_splitters import TokenTextSplitter
from langchain_core.prompts import PromptTemplate

text_splitter = TokenTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=10
)
split_docs = text_splitter.split_documents(docs)
print(f"Number of chunk created after splitting :- {len(split_docs)}")


Number of chunk created after splitting :- 9


In [7]:
from langchain_openai import OpenAIEmbeddings  
# get openai api key from platform.openai.com  
model_name = 'text-embedding-ada-002'  
embeddings = OpenAIEmbeddings(  
    model=model_name
)  


In [9]:
import os,time
from pinecone import Pinecone,ServerlessSpec

pc = Pinecone()


c:\Users\yugpa\OneDrive\Documents\AI_Summarization_project\Environment\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [10]:
index_name = 'ai-question-answering-index'  

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)
index = pc.Index(index_name)


In [11]:
split_docs


[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Parul_University', 'title': 'Parul University - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nParul University - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nHistory, gove

In [14]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(split_docs, embeddings, index_name=index_name)


In [15]:
query = "parul"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)
print(len(docs))


S It’s the state where you’ll ﬁnd Mount Rainier National Park
The Divine
ComedyBART*This epic poem by Dante is divided into 3 parts: the Inferno, the Purgatorio & the Purgatorio
RAG-T Dante’s "Inferno" is the ﬁrst part of this epic poem
RAG-S This 14th century work is divided into 3 sections: "Inferno", "Purgatorio" & "Paradiso"
For 2-way classiﬁcation, we compare against Thorne and Vlachos [57], who train RoBERTa [ 35]
to classify the claim as true or false given the gold evidence sentence. RAG achieves an accuracy
within 2.7% of this model, despite being supplied with only the claim and retrieving its own evidence.
We also analyze whether documents retrieved by RAG correspond to documents annotated as gold
evidence in FEVER. We calculate the overlap in article titles between the top kdocuments retrieved
by RAG and gold evidence annotations. We ﬁnd that the top retrieved document is from a gold article
in 71% of cases, and a gold article is present in the top 10 retrieved articles in 

In [ ]:
# index.delete(delete_all=True)


If index is already created the no need of pc

In [16]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
index_name = "ai-question-answering-index"
namespace = "Default"
vectorstore = Pinecone(
    index_name=index_name,
    embedding=OpenAIEmbeddings,
    namespace=namespace,
)

PineconeVectorStore.from_documents(split_docs, embeddings, index_name=index_name)


In [18]:
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
# Query the vectorized data

input_text = 'why the sky is blue'

vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
# Using LangChain we pass in our model for text generation.
llm=ChatGroq(temperature=0.5, model_name="llama3-70b-8192", max_tokens=512)

qa = RetrievalQA.from_chain_type(
llm=llm,
chain_type="stuff",
retriever=vectorstore.as_retriever()
)
# Finally we return the result of the search query. 
result = qa.run(input_text)
print(result)


c:\Users\yugpa\OneDrive\Documents\AI_Summarization_project\Environment\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


I don't know the answer to that question based on the provided context. The context appears to be related to natural language processing and generation models, and does not provide information about the color of the sky. If you're interested in learning about why the sky is blue, I can provide a brief explanation:

The sky appears blue because of a phenomenon called Rayleigh scattering, which is the scattering of light by small particles or molecules in the atmosphere. When sunlight enters the Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen and oxygen. These molecules scatter the light in all directions, but they scatter shorter (blue) wavelengths more than longer (red) wavelengths. This is why the sky appears blue during the daytime, as the blue light is scattered in all directions and reaches our eyes from all parts of the sky.

However, please note that this explanation is not related to the provided context, and I'm just providing it as a general knowledg